In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import pandas as pd
import numpy as np

In [10]:
train = pd.read_csv('csv_data//train.csv', index_col=0)
test = pd.read_csv('csv_data//test.csv', index_col=0)

In [11]:
train.drop(columns=['const'], inplace=True)
test.drop(columns=['const'], inplace=True)

In [13]:
train_np = np.zeros((train.num_of_action.nunique(), 30, train.shape[1] - 2))

In [51]:
X_train_np = np.zeros((314, 30, train.shape[1] - 2))
Y_train_np = np.zeros(314)
i = 0
for action in train.action.unique():
    for num in train.num_of_action.unique():
        shape_0 = train.loc[(train.num_of_action == num) & (train.action == action)].iloc[:, :-2].shape[0]
        if shape_0 != 0:
            assert shape_0 == 30, (num, action)
            X_train_np[i] = train.loc[(train.num_of_action == num) & (train.action == action)].iloc[:, :-2]
            Y_train_np[i] = train.loc[(train.num_of_action == num) & (train.action == action)].iloc[-1, -2]
            i += 1
print(i)

314


In [52]:
X_test_np = np.zeros((80, 30, test.shape[1] - 2))
Y_test_np = np.zeros(80)
i = 0
for action in test.action.unique():
    for num in test.num_of_action.unique():
        shape_0 = test.loc[(test.num_of_action == num) & (test.action == action)].iloc[:, :-2].shape[0]
        if shape_0 != 0:
            assert shape_0 == 30, (num, action)
            X_test_np[i] = test.loc[(test.num_of_action == num) & (test.action == action)].iloc[:, :-2]
            Y_test_np[i] = test.loc[(test.num_of_action == num) & (test.action == action)].iloc[-1, -2]
            i += 1
print(i)

80


In [ ]:
from tensorflow.keras.utils import to_categorical
Y_train_np = to_categorical(Y_train_np).astype(int)
Y_test_np = to_categorical(Y_test_np).astype(int)

In [76]:
np.save('csv_data/X_train_np', X_train_np)
np.save('csv_data/X_test_np', X_test_np)
np.save('csv_data/Y_train_np', Y_train_np)
np.save('csv_data/Y_test_np', Y_test_np)

In [86]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, train.shape[1] - 2)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [87]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [88]:
model.fit(X_train_np, Y_train_np, epochs=2000)

Epoch 1/2000


AttributeError: in user code:

    File "C:\Users\dandy\anaconda3\lib\site-packages\keras\engine\training.py", line 946, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\dandy\anaconda3\lib\site-packages\keras\engine\training.py", line 935, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\dandy\anaconda3\lib\site-packages\keras\engine\training.py", line 928, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\dandy\anaconda3\lib\site-packages\keras\engine\training.py", line 846, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\dandy\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py", line 532, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "C:\Users\dandy\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py", line 668, in apply_gradients
        gr